LlamaParser

In [ ]:
# LlamaIndex에서 개발한 문서 파싱 서비스
# 대규모 언어 모델(LLM)을 위해 특별히 설계되었다

# 주요 특징
# PDF, Word, Excel 등 다양한 문서 형식 지원
# 자연어 지시를 통한 맞춤형 출력 형식 제공
# 복잡한 표와 이미지 추출 기능
# JSON 모드 지원
# 외국어 지원

# LlamaParse는 독립형 API로 제공되며, LlamaCloud 플랫폼의 일부로도 사용 가능합니다. 
# 이 서비스는 문서를 파싱하고 정제하여 검색 증강 생성(RAG) 등 LLM 기반 애플리케이션의 성능을 향상시키는 것을 목표로 합니다.

# 사용자는 무료로 하루 1,000페이지를 처리할 수 있으며, 유료 플랜을 통해 추가 용량을 확보할 수 있습니다. 
# LlamaParse는 현재 공개 베타 버전으로 제공되고 있으며, 지속적으로 기능이 확장되고 있습니다.

import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

# 기본 파서 적용
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# 파서 설정
parser = Llama_parse(
    result_type = "markdown",
    num_workers = 8,
    verbose = True,
    language = "ko",
)

# SimpoerDirectoryReader 를 사용하여 파일 파싱
file_extractor = {".pdf" : parser}

documents = SimplerDirectoryReader(
    input_files = ["/SPRI_AI_Brief_2023년12월호_F.pdf"],
    file_extractor = file_extractor,
).load_data()

In [ ]:
len(documents)

In [ ]:
# 랭체인 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in documents] 

In [ ]:
docs[0].metadata

MultiModal Model로 파싱

In [ ]:
# use_vendor_multimodal_model: 멀티모달 모델 사용 여부를 지정합니다. True로 설정하면 외부 벤더의 멀티모달 모델을 사용합니다.
# vendor_multimodal_model_name: 사용할 멀티모달 모델의 이름을 지정합니다. 여기서는 "openai-gpt4o"를 사용하고 있습니다.
# vendor_multimodal_api_key: 멀티모달 모델 API 키를 지정합니다. 환경 변수에서 OpenAI API 키를 가져옵니다.
# result_type: 파싱 결과의 형식을 지정합니다. "markdown"으로 설정되어 있어 결과가 마크다운 형식으로 반환됩니다.
# language: 파싱할 문서의 언어를 지정합니다. "ko"로 설정되어 한국어로 처리됩니다.
# skip_diagonal_text: 대각선 텍스트를 건너뛸지 여부를 결정합니다.
# page_separator: 페이지 구분자를 지정할 수 있습니다.

documents = LlamaParse(
    use_vendor_multimodal_model = True,
    vendor_multimodal_model_name = "openai-gpt4o",
    vendor_multimodal_api_keys = os.environ["OPENAI_API_KEY"],
    result_type = "markdown",
    language = "ko",
    # skip_diagonal_text = True,
    # page_separator = "\n=================\n"
)

In [ ]:
# parsing 된 결과
parsed_docs = documents.load_data(file_path="./SPRI_AI_Brief_2023년12월호_F.pdf")

In [ ]:
# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]

In [ ]:
# parsing instruction 을 지정합니다.
parsing_instruction = (
    "You are parsing a brief of AI Report. Please extract tables in markdown format."
)

# LlamaParse 설정
parser = LlamaParse(
    use_vendor_multimodal_model=True,
    vendor_multimodal_model_name="openai-gpt4o",
    vendor_multimodal_api_key=os.environ["OPENAI_API_KEY"],
    result_type="markdown",
    language="ko",
    parsing_instruction=parsing_instruction,
)

# parsing 된 결과
parsed_docs = parser.load_data(file_path="data/SPRI_AI_Brief_2023년12월호_F.pdf")

# langchain 도큐먼트로 변환
docs = [doc.to_langchain_format() for doc in parsed_docs]


In [ ]:
# markdown 형식으로 추출된 테이블 확인
print(docs[-2].page_content)